In [1]:
import pandas as pd
import sqlite3

connection = sqlite3.connect('../data/checking-logs.sqlite')

In [2]:
pd.read_sql(sql="SELECT * FROM test WHERE uid='user_1';", con=connection)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136


In [27]:
test_results=pd.read_sql(sql="""
                SELECT 
                    time,
                    AVG(avg_diff) AS avg_diff
                FROM(
                SELECT 
                    'after' AS time,
                    AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
                FROM test t
                LEFT JOIN deadlines AS d ON d.labs=t.labname
                WHERE first_commit_ts > first_view_ts AND labname != 'project1'
                AND
                (SELECT COUNT(*) from test WHERE first_commit_ts < first_view_ts AND labname != 'project1' AND uid=t.uid)>0
                GROUP BY t.uid
                          
                UNION ALL         

                SELECT 
                    'before' AS time,
                    AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
                FROM test t
                LEFT JOIN deadlines AS d ON d.labs=t.labname
                WHERE first_commit_ts < first_view_ts AND labname != 'project1' 
                AND 
                (SELECT COUNT(*) from test WHERE first_commit_ts > first_view_ts AND labname != 'project1' AND uid=t.uid)>0
                GROUP BY t.uid
                )
                GROUP BY time;
            """, 
            con=connection)
test_results
#UNION
#UNION ALL

,time,avg_diff
0,after,-99.523810
1,before,-66.047619


In [30]:
control_results=pd.read_sql(sql="""
                SELECT 
                    time,
                    AVG(avg_diff) AS avg_diff
                FROM(
                SELECT 
                    'after' AS time,
                    AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
                FROM control c
                LEFT JOIN deadlines AS d ON d.labs=c.labname
                WHERE first_commit_ts > first_view_ts AND labname != 'project1'
                AND (SELECT COUNT(*) from control WHERE first_commit_ts < first_view_ts 
                            AND labname != 'project1' AND uid=c.uid)>0
                GROUP BY c.uid
                          
                UNION ALL         

                SELECT 
                    'before' AS time,
                    AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
                FROM control c
                LEFT JOIN deadlines AS d ON d.labs=c.labname
                WHERE first_commit_ts < first_view_ts AND labname != 'project1' 
                AND (SELECT COUNT(*) from control WHERE first_commit_ts > first_view_ts 
                            AND labname != 'project1' AND uid=c.uid)>0
                GROUP BY c.uid
                )
                GROUP BY time;
            """, 
            con=connection)
control_results

,time,avg_diff
0,after,-99.322222
1,before,-98.033333


In [ ]:
connection.close()

Did the hypothesis turn out to be true, and does the page affect students' behavior?

Yes